In [2]:
#reveal configuration
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'theme': 'white',
        'transition': 'none',
        'controls': 'false',
        'progress': 'true',
})

import tensorflow as tf
import numpy as np

%load_ext tikzmagic

In [3]:
%%javascript
require(['base/js/utils'],
function(utils) {
   utils.load_extensions('calico-spell-check', 'calico-document-tools', 'calico-cell-tools');
});

<IPython.core.display.Javascript object>

In [4]:
%%html
<style>
.red { color: #E41A1C; }
.orange { color: #FF7F00 }
.yellow { color: #FFC020 }         
.green { color: #4DAF4A }                  
.blue { color: #377EB8; }
.purple { color: #984EA3 }       
         
#for reveal         
.aside .controls, .reveal .controls {
    display: none !important;                            
    width: 0px !important;
    height: 0px !important;
}
    
.rise-enabled .reveal .slide-number {
    right: 25px;
    bottom: 25px;                        
    font-size: 200%;     
    color: #377EB8;                        
}         
         
.rise-enabled .reveal .progress span {
    background: #377EB8;
}     
         
.present .top {
    position: fixed !important;
    top: 0 !important;                                   
}                
</style>

# Deep Learning for Natural Language Processing

In this chapter we will look into applications of Recurrent Neural Networks (RNNs) for NLP. Deep Learning and RNNs are currently used in many tasks, for instance, machine translation, machine comprehension, question answering, text summarization and generation. In this chapter we take recognizing textual entailment (RTE) as an example task and further look into several tools that help training RNNs for this task.

# Outline
- Use-case: Recognizing Textual Entailment
  - Conditional Encoding
  - Attention
- Bag of Tricks
  - Continuous Optimization   
  - Regularization  
  - Hyper-parameter Optimization
  - Pre-trained Representations
  - Batching
  - Bucketing
  - <span class="blue">TensorFlow</span> `dynamic_rnn`  
  - Bi-directional RNNs

# Recognizing Textual Entailment (RTE)

RTE is the task of determining the logical relationship between two natural language sentences. Although this can be simply seen as a classification problem, it is a very difficult task as it needs commonsense and background knowledge about the world, some understanding of natural language, as well a form of fine-grained reasoning.

- **A wedding party is taking pictures**
  - There is a funeral					: **<span class=red>Contradiction</span>**
  - They are outside					: **<span class=blue>Neutral</span>**
  - Someone got married				    : **<span class=green>Entailment</span>**

### State of the Art until 2015

[<span class=blue>Lai and Hockenmaier, 2014, Jimenez et al., 2014, Zhao et al., 2014, Beltagy et al., 2015</span> etc.]

- Engineered natural language processing pipelines
- Various external resources
- Specialized subcomponents
- Extensive manual creation of **features**:
  - Negation detection, word overlap, part-of-speech tags, dependency parses, alignment, unaligned matching, chunk alignment, synonym, hypernym, antonym, denotation graph

### Neural Networks for RTE

As shown above, models for RTE heavily relied on engineered features. One reason why neural networks were not applied to this task is due to the absence of high-quality large-scale RTE corpora.

**Previous RTE corpora**:
- Tiny data sets (1k-10k examples)
- Partly synthetic examples

**Stanford Natural Inference Corpus (SNLI)**:
- 500k sentence pairs
- Two orders of magnitude larger than existing RTE data set
- All examples generated by humans


### Independent Sentence Encoding

With the introduction of a large-scale RTE corpora, neural networks became feasible to train. A first baseline model by Bowman et al. (2015) used LSTMs to encode the premise and hypothesis independently of each other.

[<span class=blue>Bowman et al, 2015</span>]

Same LSTM encodes premise and hypothesis

<img src="dl-applications-figures/rte.svg" width=800/> 

### Independent Sentence Encoding

The way RNNs can encode a sentence is to apply the RNN to the sequence of tokens and subsequently take the last output vector as the representation of the sentence. After training the RNN, the hope is that this last output vector would capture the semantics of the sentence that is needed for solving the downstream task (RTE in this case).

[<span class=blue>Bowman et al, 2015</span>]

Same LSTM encodes premise and hypothesis

<img src="dl-applications-figures/rte_encoding.svg" width=800/> 

> You can’t cram the meaning of a whole
%&!\$# sentence into a single \$&!#* vector!
>
> -- <cite>Raymond J. Mooney</cite>

The problem with this approach is that encoding a whole sentence into a single dense vector has obvious representational limitations. However, it is a simple model that sometimes works surprisingly well in practice and often serves as a baseline for more complex architectures.

### Independent Sentence Encoding

Once both sentences are encoded as vectors, we can use a multi-layer perceptron followed by a softmax to model the probability of each class (entailment, neutral, contradiction) given the two sentences.

[<span class=blue>Bowman et al, 2015</span>]

<img src="dl-applications-figures/mlp.svg" width=700/> 

#### <span class="blue">TensorFlow</span>: Multi-layer Perceptron

Below we show simple TensorFlow code for a multi-layer perceptron. Note that we sample two random sentence representations, whereas in practice we would use the output of an RNN as explained earlier.

In [289]:
with tf.Graph().as_default():
    def mlp(input_vector, layers=3, hidden_dim=200, output_dim=3):    
        # [input_size] => [input_size x 1] (column vector)
        tmp = tf.expand_dims(input_vector, 1)
        for i in range(layers+1):
            W = tf.get_variable(
                "W_"+str(i), [hidden_dim, hidden_dim])
            # tanh(Wx^T)
            tmp = tf.tanh(tf.matmul(W, tmp))
        W = tf.get_variable(
            "W_"+str(layers+1), [output_dim, hidden_dim])
        # [input_size x 1] => [input_size]
        return tf.squeeze(tf.matmul(W, tmp))
    premise = tf.placeholder(tf.float32, [None], "premise")
    hypothesis = tf.placeholder(tf.float32, [None], "hypothesis")
    output = tf.nn.softmax(mlp(tf.concat(0, [premise, hypothesis])))
    # in practice: outputs of an LSTM
    v1 = np.random.rand(100); v2 = np.random.rand(100)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print(sess.run(output, {premise: v1, hypothesis: v2}))

[ 0.97730732  0.02076106  0.00193162]


## Results

| Model | k | θ<sub>W+M</sub> | θ<sub>M</sub> | Train | Dev | Test |
|-|-|-|-|-|-|-|
| LSTM [<span class=blue>Bowman et al.</span>] | 100 | \\(\approx\\)10M | 221k | 84.4 | - | 77.6|
| Classifier [<span class=blue>Bowman et al.</span>]| - | - | - | 99.7 | - | 78.2|

In fact, this simple way of encoding the relationship between two sentences is still inferior compared to a classifier with hand-engineered features. Next, we will investigate slightly more complex models.

### Conditional Endcoding

<img src="dl-applications-figures/conditional.svg" width=800/> 

If we think more about the problem, it does not seem natural to encode both sentences independently. Instead, the way we read the hypothesis could be influenced by our understanding of the premise. For designing a model we could thus use two LSTMs, one for the premise and one for the hypothesis, where the initial state of the hypothesis model is initialized by the hidden state of the premise model.

\begin{align}
\text{softmax}(\text{tanh}(\mathbf{W}\mathbf{h}_N))
\end{align}

Instead of a concatenation of the premise and hypothesis representation, we now simply project the last output representation into the space of the three target classes and apply the softmax function.

### Conditional Endcoding

<img src="dl-applications-figures/conditional_encoding.svg" width=800/> 

This has the advantage that we do not need to encode both sentences as fixed vector representations anymore. While the premise still needs to be encoded (last hidden state, $\mathbf{c}_5$), the LSTM used for the hypothesis can use all its parameters to track the state of the sequence comparison for the final prediction. 

\begin{align}
\text{softmax}(\text{tanh}(\mathbf{W}\mathbf{h}_N))
\end{align}

## Results

| Model | k | θ<sub>W+M</sub> | θ<sub>M</sub> | Train | Dev | Test |
|-|-|-|-|-|-|-|
| LSTM [<span class=blue>Bowman et al.</span>] | 100 | \\(\approx\\)10M | 221k | 84.4 | - | 77.6|
| Classifier [<span class=blue>Bowman et al.</span>]| - | - | - | 99.7 | - | 78.2|
| Conditional Endcoding | 159 | 3.9M | 252k | 84.4 | 83.0 | 81.4|

### Attention [<span class=blue>Graves 2013</span>, <span class=blue>Bahdanau et al. 2015</span>]


<img src="dl-applications-figures/attention.svg" width=800/> 

<div class=small>
\begin{align}
  \mathbf{M} &= \tanh(\mathbf{W}^y\mathbf{Y}+ \mathbf{W}^h\mathbf{h}_N\mathbf{1}^T_L)&\mathbf{M}&\in\mathbb{R}^{k \times L}\\
  \alpha &= \text{softmax}(\mathbf{w}^T\mathbf{M})&\alpha&\in\mathbb{R}^L\\
  \mathbf{r} &= \mathbf{Y}\alpha^T&\mathbf{r}&\in\mathbb{R}^k
\end{align}
</div>

### Attention [<span class=blue>Graves 2013</span>, <span class=blue>Bahdanau et al. 2015</span>]

<img src="dl-applications-figures/attention_encoding.svg" width=800/> 

<div class=small>
\begin{align}
  \mathbf{M} &= \tanh(\mathbf{W}^y\mathbf{Y}+ \mathbf{W}^h\mathbf{h}_N\mathbf{1}^T_L)&\mathbf{M}&\in\mathbb{R}^{k \times L}\\
  \alpha &= \text{softmax}(\mathbf{w}^T\mathbf{M})&\alpha&\in\mathbb{R}^L\\
  \mathbf{r} &= \mathbf{Y}\alpha^T&\mathbf{r}&\in\mathbb{R}^k
\end{align}
</div>

<img  src="./dl-applications-figures/camel.png"/>

#### Contextual Understanding

<img  src="./dl-applications-figures/pink.png"/>

## Results

| Model | k | θ<sub>W+M</sub> | θ<sub>M</sub> | Train | Dev | Test |
|-|-|-|-|-|-|-|
| LSTM [<span class=blue>Bowman et al.</span>] | 100 | \\(\approx\\)10M | 221k | 84.4 | - | 77.6|
| Classifier [<span class=blue>Bowman et al.</span>]| - | - | - | 99.7 | - | 78.2|
| Conditional Encoding | 159 | 3.9M | 252k | 84.4 | 83.0 | 81.4|
| Attention | 100 | 3.9M | 242k | 85.4 | 83.2 | 82.3 |

#### Fuzzy Attention

<img  src="./dl-applications-figures/mimes.png"/>

### Word-by-word Attention [<span class=blue>Bahdanau et al. 2015</span>, <span class=blue>Hermann et al. 2015</span>, <span class=blue>Rush et al. 2015</span>]

<img src="dl-applications-figures/word_attention.svg" width=800/> 

<div class=small>
\begin{align}
  \mathbf{M}_t &= \tanh(\mathbf{W}^y\mathbf{Y}+(\mathbf{W}^h\mathbf{h}_t+\mathbf{W}^r\mathbf{r}_{t-1})\mathbf{1}^T_L) & \mathbf{M}_t &\in\mathbb{R}^{k\times L}\\
  \alpha_t &= \text{softmax}(\mathbf{w}^T\mathbf{M}_t)&\alpha_t&\in\mathbb{R}^L\\
  \mathbf{r}_t &= \mathbf{Y}\alpha^T_t + \tanh(\mathbf{W}^t\mathbf{r}_{t-1})&\mathbf{r}_t&\in\mathbb{R}^k
\end{align}
</div>

### Word-by-word Attention [<span class=blue>Bahdanau et al. 2015</span>, <span class=blue>Hermann et al. 2015</span>, <span class=blue>Rush et al. 2015</span>]

<img src="dl-applications-figures/word_attention_encoding.svg" width=800/> 

<div class=small>
\begin{align}
  \mathbf{M}_t &= \tanh(\mathbf{W}^y\mathbf{Y}+(\mathbf{W}^h\mathbf{h}_t+\mathbf{W}^r\mathbf{r}_{t-1})\mathbf{1}^T_L) & \mathbf{M}_t &\in\mathbb{R}^{k\times L}\\
  \alpha_t &= \text{softmax}(\mathbf{w}^T\mathbf{M}_t)&\alpha_t&\in\mathbb{R}^L\\
  \mathbf{r}_t &= \mathbf{Y}\alpha^T_t + \tanh(\mathbf{W}^t\mathbf{r}_{t-1})&\mathbf{r}_t&\in\mathbb{R}^k
\end{align}
</div>

#### Reordering

<img src="./dl-applications-figures/reordering.png" width=60%/>

#### Garbage Can = Trashcan

<img  src="./dl-applications-figures/trashcan.png" width=90%/>

#### Kids =  Girl + Boy

<img  src="./dl-applications-figures/kids.png" width=80%/>

## Snow is outside

<img  src="./dl-applications-figures/snow.png"/>

## Results

| Model | k | θ<sub>W+M</sub> | θ<sub>M</sub> | Train | Dev | Test |
|-|-|-|-|-|-|-|
| LSTM [<span class=blue>Bowman et al.</span>] | 100 | \\(\approx\\)10M | 221k | 84.4 | - | 77.6|
| Classifier [<span class=blue>Bowman et al.</span>]| - | - | - | 99.7 | - | 78.2|
| Conditional Encoding | 159 | 3.9M | 252k | 84.4 | 83.0 | 81.4|
| Attention | 100 | 3.9M | 242k | 85.4 | 83.2 | 82.3 |
| Word-by-word Attention | 100 | 3.9M | 252k | 85.3 | **83.7** | **83.5** |

# Bag of Tricks

## Continuous Optimization

<img  src="https://upload.wikimedia.org/wikipedia/commons/6/68/Gradient_ascent_%28surface%29.png" width=800/>
Source: Wikipedia

### Momentum

<img  src="http://sebastianruder.com/content/images/2016/09/saddle_point_evaluation_optimizers.gif" width=800/>

#### <span class="blue">TensorFlow</span>: Optimizers

- GradientDescent
- RMSProp
- Adagrad

- Adadelta
- **Adam**

In [296]:
with tf.Graph().as_default():
    x = tf.get_variable("param", [])  
    loss = -tf.log(tf.sigmoid(x))  # dummy example
    optim = tf.train.AdamOptimizer(learning_rate=0.1)
    min_op = optim.minimize(loss)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        sess.run(x.assign(1.5))
        for i in range(10):
            print(sess.run([min_op, loss], {})[1])

0.201413
0.183901
0.167832
0.153134
0.139727
0.127529
0.116455
0.106421
0.0973447
0.0891449


### Gradient Clipping

<img src="rnn-figures/error_surface.png"></img>

In [297]:
with tf.Graph().as_default():
    x = tf.get_variable("param", [])      
    loss = -tf.log(tf.sigmoid(x))  # dummy example
    optim = tf.train.AdamOptimizer(learning_rate=0.1)    
    gradients = optim.compute_gradients(loss)
    capped_gradients = \
        [(tf.clip_by_value(grad, -0.1, 0.1), var) for grad, var in gradients]
    min_op = optim.apply_gradients(capped_gradients)        
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables()); sess.run(x.assign(1.5))        
        for i in range(100):
            if i % 10 == 0:
                grads = sess.run([min_op, gradients, capped_gradients], {})[1:]
                print(" => ".join([str(grad[0][0]) for grad in grads]))

-0.182426 => -0.1
-0.167982 => -0.1
-0.154465 => -0.1
-0.141851 => -0.1
-0.130109 => -0.1
-0.119203 => -0.1
-0.109097 => -0.1
-0.0997508 => -0.0997508
-0.0911243 => -0.0911243
-0.0832129 => -0.0832129


## Regularization

### Dropout

<img src="http://everglory99.github.io/Intro_DL_TCC/intro_dl_images/dropout1.png" width=1000></img>

For RNNs: `tf.nn.rnn_cell.DropoutWrapper`

In [22]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, [None], "input")
    x_dropped = tf.nn.dropout(x, 0.7)  # keeps 70% of values
    
    with tf.Session() as sess:        
        print(sess.run(x_dropped, {x: np.random.rand(6)}))

[ 0.19594376  0.55282122  0.05783162  0.          0.54578322  0.        ]


### Early Stopping

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1f/Overfitting_svg.svg" width=800></img>

## Hyper-parameter Optimization
[<span class=blue>Bergstra and Bengio 2012</span>]
<img src="https://cdn-images-1.medium.com/max/800/1*ZTlQm_WRcrNqL-nLnx6GJA.png"></img>

## Pre-trained Representations

In [39]:
with tf.Graph().as_default():
    vocab_size = 4; embedding_size = 3
    W = tf.get_variable("W", [vocab_size, embedding_size], trainable=False)
    W = W.assign(np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]))
    seq =  tf.placeholder(tf.int64, [None], "seq")
    seq_embedded = tf.nn.embedding_lookup(W, seq)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print(sess.run(seq_embedded, {seq:[0, 3, 2, 3, 1]})[2])

[ 6.  7.  8.]


### Projecting Representations

In [38]:
with tf.Graph().as_default():
    vocab_size = 4; embedding_size = 3; input_size = 2
    W = tf.get_variable("W", [vocab_size, embedding_size], trainable=False)
    W = W.assign(np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]))
    seq =  tf.placeholder(tf.int64, [None], "seq")
    seq_embedded = tf.contrib.layers.linear(tf.nn.embedding_lookup(W, seq), input_size)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print(sess.run(seq_embedded, {seq:[0, 3, 2, 3, 1]})[2])

[ -4.09411907 -14.17657852]


## Batching
- Utilization of GPUs

<img src="dl-applications-figures/batching.svg" width=800 />

## Bucketing
- Increased computational efficiency by grouping data into buckets of similar length

<img src="dl-applications-figures/bucketing.svg" width=500 />

## <span class="blue">TensorFlow</span>: `dynamic_rnn`
- Avoids padding (which can improve performance)

<img src="dl-applications-figures/dynamic.svg" width=600 />

In [41]:
with tf.Graph().as_default():
    input_size = 2; output_size = 3; batch_size = 5; max_length = 7
    cell = tf.nn.rnn_cell.LSTMCell(output_size)
    input_embedded = tf.placeholder(tf.float32, [None, None, input_size], "input_embedded")
    input_length = tf.placeholder(tf.int64, [None], "input_length")
    outputs, states = \
        tf.nn.dynamic_rnn(cell, input_embedded, sequence_length=input_length, dtype=tf.float32)    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print(sess.run(states, {
                    input_embedded: np.random.randn(batch_size, max_length, input_size), 
                    input_length: np.random.randint(1, max_length, batch_size)
                }).h)

[[-0.07828863  0.07784089 -0.00747216]
 [-0.038492    0.03572574 -0.03601231]
 [-0.03372879 -0.09718083  0.03966172]
 [-0.18546213  0.10253921 -0.0991728 ]
 [ 0.06936012 -0.20678709 -0.01240843]]


## Bi-directional RNNs
`tf.nn.bidirectional_dynamic_rnn`

<img src="dl-applications-figures/birnn.svg" width=1000 />

# Background Material

- [A Primer on Neural Network Models for Natural Language Processing](http://www.jair.org/media/4992/live-4992-9623-jair.pdf)
- [Stanford Course: Deep Learning for Natural Language Processing](http://cs224d.stanford.edu/syllabus.html)
- [Nando de Freitas' Deep Learning Lecture](https://www.youtube.com/watch?v=dV80NAlEins&list=PLE6Wd9FR--EfW8dtjAuPoTuPcqmOV53Fu)
- [Deep Learning Book by Goodfellow et al.](http://www.deeplearningbook.org/)